A cute little demo showing the simplest usage of minGPT. Configured to run fine on Macbook Air in like a minute.

In [7]:
import torch
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from mingpt.utils import set_seed

set_seed(3407)

In [5]:
import random

def random_mult_instance(length):
    a = [random.randint(0,9) for i in range(length)]
    b = [random.randint(0,9) for i in range(length)]

    val_a = int(''.join(str(d) for d in a))
    val_b = int(''.join(str(d) for d in b))

    val_c = val_a * val_b

    str_c = str(val_c)
    need_len = 2*length
    str_c = (need_len - len(str_c)) * '0' + str_c
    return a + b + [int(d) for d in str_c]

random_mult_instance(1)


[1, 4, 0, 4]

In [8]:
class MultDataset(Dataset):
    """ 
    Dataset for the Add problem. E.g. for problem length 3:
    12 + 333 = 345
    Input: 0 1 2 3 3 3 -> Output: 0 3 4 5
    Which will feed into the transformer concatenated as:
    input:  0 1 2 3 3 3 0 3 4
    output: I I I I I 0 3 4 5
    where I is "ignore", as the transformer is reading the input sequence
    """

    def __init__(self, split, length=3):
        assert split in {'train', 'test'}
        self.split = split
        self.length = length
    
    def __len__(self):
        return 10000 # ...
    
    def get_vocab_size(self):
        return 10
    
    def get_block_size(self):
        return 4 * self.length

    def __getitem__(self, idx):
        while True:
            rai = random_mult_instance(self.length)
            h = hash(str(rai[:2*self.length]))
            
            inp_split = 'test' if h % 4 == 0 else 'train' # designate 25% of examples as test
            if inp_split == self.split:
                break # ok
        
        x = torch.tensor(rai[:-1], dtype=torch.long)
        y = torch.tensor(rai[1:], dtype=torch.long)
        
        # we only want to predict at output locations, mask out the loss at the input locations
        y[:2*self.length-1] = -1
        return x, y

In [9]:
# print an example instance of the dataset
train_dataset = MultDataset('train')
test_dataset = MultDataset('test')
x, y = train_dataset[0]

print (x)
for a, b in zip(x,y):
    print(int(a),int(b))

tensor([0, 6, 2, 5, 3, 6, 0, 3, 3, 2, 3])
0 -1
6 -1
2 -1
5 -1
3 -1
6 0
0 3
3 3
3 2
2 3
3 2


In [17]:
# create a GPT instance
from mingpt.model import GPT

model_config = GPT.get_default_config()
model_config.model_type = 'gpt-micro'
#model_config.model_type = 'gpt-nano'

model_config.vocab_size = train_dataset.get_vocab_size()
model_config.block_size = train_dataset.get_block_size()
model = GPT(model_config)

number of parameters: 0.80M


In [18]:
print (model_config.n_head, model_config.n_layer, model_config.n_embd)

4 4 128


In [19]:
# create a Trainer object
from mingpt.trainer import Trainer

train_config = Trainer.get_default_config()
train_config.learning_rate = 5e-4 # the model we're using is so small that we can go a bit faster
train_config.max_iters = 30000
train_config.num_workers = 0
trainer = Trainer(train_config, model, train_dataset)

running on device cpu


In [20]:
def batch_end_callback(trainer):
    if trainer.iter_num % 100 == 0:
        print(f"iter_dt {trainer.iter_dt * 1000:.2f}ms; iter {trainer.iter_num}: train loss {trainer.loss.item():.5f}")
trainer.set_callback('on_batch_end', batch_end_callback)

trainer.run()

iter_dt 0.00ms; iter 0: train loss 2.32302
iter_dt 52.64ms; iter 100: train loss 1.92707
iter_dt 52.89ms; iter 200: train loss 1.79622
iter_dt 52.57ms; iter 300: train loss 1.71167
iter_dt 54.56ms; iter 400: train loss 1.67194
iter_dt 52.17ms; iter 500: train loss 1.58977
iter_dt 52.79ms; iter 600: train loss 1.61766
iter_dt 53.42ms; iter 700: train loss 1.62040
iter_dt 54.04ms; iter 800: train loss 1.54085
iter_dt 54.10ms; iter 900: train loss 1.56250
iter_dt 53.92ms; iter 1000: train loss 1.57536
iter_dt 53.52ms; iter 1100: train loss 1.56704
iter_dt 53.87ms; iter 1200: train loss 1.54073
iter_dt 51.74ms; iter 1300: train loss 1.60379
iter_dt 51.79ms; iter 1400: train loss 1.48073
iter_dt 54.57ms; iter 1500: train loss 1.59941
iter_dt 54.87ms; iter 1600: train loss 1.50578
iter_dt 54.58ms; iter 1700: train loss 1.53797
iter_dt 54.74ms; iter 1800: train loss 1.46779
iter_dt 53.70ms; iter 1900: train loss 1.41086
iter_dt 55.24ms; iter 2000: train loss 1.44683
iter_dt 54.74ms; iter 2100

In [21]:
# now let's perform some evaluation
model.eval()
None

In [23]:
def eval_add_split(trainer, split, max_batches):
    dataset = {'train':train_dataset, 'test':test_dataset}[split]
    n = train_dataset.length # naugy direct access shrug
    results = []
    mistakes_printed_already = 0
    loader = DataLoader(dataset, batch_size=100, num_workers=0, drop_last=False)
    #loader = DataLoader(dataset, batch_size=1, num_workers=0, drop_last=False)
    for b, (x, y) in enumerate(loader):
        x = x.to(trainer.device)
        y = y.to(trainer.device)

        inp = x[:, :2*n]
        sol = y[:, -2*n:]
        
        cat = model.generate(inp, 2*n, do_sample=False) # using greedy argmax, not sampling
        sol_candidate = cat[:, -2*n:]
        correct = (sol == sol_candidate).all(1).cpu() 
        for i in range(x.size(0)):
            results.append(int(correct[i]))
    
    rt = torch.tensor(results, dtype=torch.float)
    print("%s final score: %d/%d = %.2f%% correct" % (split, rt.sum(), len(results), 100*rt.mean()))
    return rt.sum()

# run a lot of examples from both train and test through the model and verify the output correctness
with torch.no_grad():
    train_score = eval_add_split(trainer, 'train', max_batches=50)
    test_score  = eval_add_split(trainer, 'test',  max_batches=50)

train final score: 254/10000 = 2.54% correct
test final score: 245/10000 = 2.45% correct
